In [2]:
a = 0xffffffff
b = 1365

print(hex(a ^ b))

0xfffffaaa


In [16]:
from eth.chains.base import MiningChain
from eth.db.atomic import AtomicDB
from eth.vm.forks import LATEST_VM  # 或 BerlinTransaction 等，根据 fork
from eth_utils import to_canonical_address
from eth_account import Account
from eth_keys import keys
from eth_utils import decode_hex, keccak, to_checksum_address
from eth_account import Account

# 创建一个新的账户及私钥
new_account = Account.create("20250323")
print(type(new_account.address))
print(new_account.key)

<class 'str'>
b'\x92\x87\xda0[\xb9\\i\xc3\xb9\xfb\xdfx\xb3\xaak\xa2\xf5\xf1\x91\x92\xa1j\xd2I@\x7f\xecJ{6\xa3'


In [ ]:
from eth.chains.base import MiningChain
from eth.db.atomic import AtomicDB
from eth_utils import to_canonical_address

# 定义创世区块的参数
genesis_params = {
    "chain_id": 1337,  # 设定链 ID
    "difficulty": 1,
    "gas_limit": 10**7,
    "timestamp": 1735377287,  # 设定创世时间戳
}

# 设置初始账户状态
genesis_state = {
    to_canonical_address("0x5ff13d1ff093c87bcc6b6ece07afbea94b684dcb"): {
        "balance": 10**18,
        "nonce": 0,
        "code": b"",
        "storage": {}
    },
    to_canonical_address("0x32353a6c91143bfd6c7d363b546e62a9a2489a20"): {
        "balance": 5 * 10**18,
        "nonce": 0,
        "code": b"",
        "storage": {}
    }
}

# 创建本地测试链
db = AtomicDB()
chain = MiningChain.from_genesis(db, genesis_params, genesis_state)

# 获取 VM 并查看创世块
vm = chain.get_vm()
print("创世块:", chain.get_canonical_head())


AttributeError: Chain classes must define the VMs in vm_configuration

In [4]:
import sys

class SilentFileWriter:
    def __init__(self, filename):
        self.file = open(filename, 'w', encoding='utf-8')
        self.original_stdout = sys.stdout  # 备份原输出流

    def write(self, text):
        # 仅写入文件，不输出到控制台
        self.file.write(text)

    def flush(self):
        # 确保缓冲区内容写入文件
        self.file.flush()

# 使用示例：静默重定向所有输出到文件
sys.stdout = SilentFileWriter('output.log')

# 假设这是第三方包的代码
def third_party_function():
    print("这是来自第三方包的输出（控制台不会显示）")

third_party_function()

# 程序结束后恢复原输出流（可选）
sys.stdout = sys.stdout.original_stdout

In [1]:
import rlp
from eth_keys import keys
from eth_utils import decode_hex, keccak, to_checksum_address
from eth_typing import Address
from eth import constants
from eth.chains.base import MiningChain
from eth.consensus.pow import mine_pow_nonce
from eth.vm.forks.byzantium import ByzantiumVM
from eth.vm.forks.frontier import FrontierVM
from eth.db.atomic import AtomicDB
from eth.vm.forks.frontier.opcodes import FRONTIER_OPCODES

# 计算合约地址
def compute_contract_address(sender: bytes, nonce: int) -> bytes:
    return keccak(rlp.encode([sender, nonce]))[-20:]

# 1. 定义创世区块
GENESIS_PARAMS = {
    'difficulty': 1,
    'gas_limit': 3141592,
    'timestamp': 1514764800,  # 方便复现
}

# 2. 生成私钥和发送者地址
SENDER_PRIVATE_KEY = keys.PrivateKey(
    decode_hex('0x45a915e4d060149eb4365960e6a7a45f334393093061116b197e3240065ff2d8')
)
# 私钥不能泄露，所以包装一下
SENDER = Address(SENDER_PRIVATE_KEY.public_key.to_canonical_address())

# 3. 智能合约字节码
contract_bytecode = (
    "6080604052600436106100405763ffffffff7c010000000000000000000000000000000000000000000000000000000060003504166379c76e1a8114610045575b600080fd5b34801561005157600080fd5b5061007373ffffffffffffffffffffffffffffffffffffffff60043516610075565b005b60005473ffffffffffffffffffffffffffffffffffffffff1633146100fb57604080517f08c379a000000000000000000000000000000000000000000000000000000000815260206004820152601360248201527f556e617574686f72697a65642063616c6c657200000000000000000000000000604482015290519081900360640190fd5b60008054604080517f70a08231000000000000000000000000000000000000000000000000000000008152306004820152905173ffffffffffffffffffffffffffffffffffffffff8086169463a9059cbb949091169285926370a0823192602480840193602093929083900390910190829087803b15801561017c57600080fd5b505af1158015610190573d6000803e3d6000fd5b505050506040513d60208110156101a657600080fd5b5051604080517c010000000000000000000000000000000000000000000000000000000063ffffffff861602815273ffffffffffffffffffffffffffffffffffffffff9093166004840152602483019190915251604480830192600092919082900301818387803b15801561021a57600080fd5b505af115801561022e573d6000803e3d6000fd5b50505050505600a165627a7a72305820300b43bef296fcf41f342266bed518123eab89057e6fc9c4fcea47cd754787bf0029"
)

# 4. 配置链，使用 FrontierVM
TestChain = MiningChain.configure(
    __name__="TestChain",
    vm_configuration=((constants.GENESIS_BLOCK_NUMBER, ByzantiumVM),),
)
chain = TestChain.from_genesis(AtomicDB(), GENESIS_PARAMS)

# 5. 获取 VM 实例
vm = chain.get_vm()
vm.state.enanble_tracing = True

# 6. 获取 nonce 并创建部署交易
# 获取下一个nonce
deploy_nonce = vm.state.get_nonce(SENDER)
tx_deploy = vm.create_unsigned_transaction(
    nonce=deploy_nonce,    # 交易序号
    gas_price=0,          # Gas 单价（单位：wei）
    gas=3000000,          # Gas 上限
    to=b'',               # 空地址（表示合约创建）
    value=0,              # 转账金额（单位：wei）
    data=bytes.fromhex(contract_bytecode),  # 合约字节码
)
# 进行交易签名
signed_tx_deploy = tx_deploy.as_signed_transaction(SENDER_PRIVATE_KEY)

# 7. 执行交易，获取计算结果
# 将带有合约的交易部署到区块链上执行
print("\n=== 合约部署执行情况 ===")
computation = vm.state.apply_transaction(signed_tx_deploy)

# 8. 打印 opcode 执行日志
opcode_map = {v.mnemonic: k for k, v in FRONTIER_OPCODES.items()}

# 9. 获取合约输出
print("\n合约执行返回的结果:", computation._output.hex())

# 10. 计算合约地址
contract_address = compute_contract_address(SENDER, deploy_nonce)
print("\n部署成功，合约地址为：", to_checksum_address(contract_address))



=== 合约部署执行情况 ===
OPCODE: 0x60 (PUSH1) | pc: 0 | stack: []
OPCODE: 0x60 (PUSH1) | pc: 2 | stack: ['0x80']
OPCODE: 0x52 (MSTORE) | pc: 4 | stack: ['0x80', '0x40']
OPCODE: 0x60 (PUSH1) | pc: 5 | stack: []
OPCODE: 0x36 (CALLDATASIZE) | pc: 7 | stack: ['0x04']
OPCODE: 0x10 (LT) | pc: 8 | stack: ['0x04', '0x0']
OPCODE: 0x61 (PUSH2) | pc: 9 | stack: ['0x1']
OPCODE: 0x57 (JUMPI) | pc: 12 | stack: ['0x1', '0x0040']
OPCODE: 0x5b (JUMPDEST) | pc: 64 | stack: []
OPCODE: 0x60 (PUSH1) | pc: 65 | stack: []
OPCODE: 0x80 (DUP1) | pc: 67 | stack: ['0x00']
OPCODE: 0xfd (REVERT) | pc: 68 | stack: ['0x00', '0x00']

合约执行返回的结果: 

部署成功，合约地址为： 0x6295eE1B4F6dD65047762F924Ecd367c17eaBf8f


In [5]:
from eth_keys import keys
from eth_utils import decode_hex
from eth_typing import Address
from eth import constants
from eth.chains.base import MiningChain
from eth.consensus.pow import mine_pow_nonce
from eth.vm.forks.byzantium import ByzantiumVM
from eth.db.atomic import AtomicDB


GENESIS_PARAMS = {
    'difficulty': 1,
    'gas_limit': 3141592,
    # We set the timestamp, just to make this documented example reproducible.
    # In common usage, we remove the field to let py-evm choose a reasonable default.
    'timestamp': 1514764800,
}

SENDER_PRIVATE_KEY = keys.PrivateKey(
    decode_hex('0x45a915e4d060149eb4365960e6a7a45f334393093061116b197e3240065ff2d8')
)

SENDER = Address(SENDER_PRIVATE_KEY.public_key.to_canonical_address())

RECEIVER = Address(b'\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\x02')

klass = MiningChain.configure(
    __name__='TestChain',
    vm_configuration=(
        (constants.GENESIS_BLOCK_NUMBER, ByzantiumVM),
    ))

chain = klass.from_genesis(AtomicDB(), GENESIS_PARAMS)
genesis = chain.get_canonical_block_header_by_number(0)
vm = chain.get_vm()

nonce = vm.state.get_nonce(SENDER)

tx = vm.create_unsigned_transaction(
    nonce=nonce,
    gas_price=0,
    gas=100000,
    to=RECEIVER,
    value=0,
    data=b'',
)

signed_tx = tx.as_signed_transaction(SENDER_PRIVATE_KEY)

chain.apply_transaction(signed_tx)

# Normally, we can let the timestamp be chosen automatically, but
# for the sake of reproducing exactly the same block every time,
# we will set it manually here:
chain.set_header_timestamp(genesis.timestamp + 1)

# We have to finalize the block first in order to be able read the
# attributes that are important for the PoW algorithm
block_result = chain.get_vm().finalize_block(chain.get_block())
block = block_result.block

# based on mining_hash, block number and difficulty we can perform
# the actual Proof of Work (PoW) mechanism to mine the correct
# nonce and mix_hash for this block
nonce, mix_hash = mine_pow_nonce(
    block.number,
    block.header.mining_hash,
    block.header.difficulty,
)  
# (takes too long for doctest to process)

chain.mine_block(mix_hash=mix_hash, nonce=nonce) 

Exception: Too many attempts at POW mining, giving up

In [40]:
from eth.vm.forks.frontier import FrontierVM

class TracingFrontierVM(FrontierVM):
    computation_class = TracingFrontierComputation


In [41]:
TestChain = MiningChain.configure(
    __name__="TestChain",
    vm_configuration=((constants.GENESIS_BLOCK_NUMBER, TracingFrontierVM),),
)


In [42]:
# 执行交易，获取计算结果
computation = vm.state.apply_transaction(signed_tx_deploy)

# 输出追踪日志
print("\n=== 合约执行 trace ===")
for step in computation.trace:
    print(f"PC: {step['pc']}, Opcode: {step['mnemonic']} (0x{step['opcode']:02x}), "
          f"Stack: {step['stack']}, Memory (len): {len(step['memory'])}, Gas Left: {step['gas_left']}")


ValidationError: Invalid transaction nonce: Expected 1, but got 0